In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import warnings
from sklearn import metrics
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import os
import pdb
import pickle
import sys
#sys.path.append("../..")
import argparse
from sklearn.preprocessing import OneHotEncoder
import warnings 
import operator
warnings.filterwarnings('ignore')
import scipy as sp

In [32]:
train_data=pd.read_csv('traindata.csv')

In [28]:
train_data['couple1'].describe()

count    39601.000000
mean         0.171334
std          0.376805
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: couple1, dtype: float64

In [22]:
train_data_preprocess['couple1'].describe()

count    38850.000000
mean         0.170991
std          0.376506
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: couple1, dtype: float64

In [33]:
train_data_preprocess,train_data_user_id=data_preprocession('traindata.csv',Train=True) 

In [18]:
train_data_user_id.head()

0    5668
1    9094
2    5999
3    5592
4    7133
Name: user_id, dtype: int64

In [137]:
def data_preprocessing(filename,Train):
    data=pd.read_csv(os.path.join(os.getcwd(),filename))
    
    ### categorize the variable
    def cat_booking_hour(x):
        return 'On_Demand' if x <= 3 else "Adv_Booked"
    
    def booking_interval(x):
        if x<0:
            return 0
        elif x<=180:
            return x
        else:
            return 180
    

    def cat_client_rank(x):
        return 1 if x in ["AAA", "AA", "A"] else 0

    def cat_city_maturity(x):
        if x in [2,5,3,4,13,12,9,1,14,15,17,18,10,16]:
            return 'Grandfather_Market'
        elif x in [20,22,24,25,21,23,27,29,30,31,32,34,53,35]:
            return 'Mature_Market'
        elif x in [57,56,58,38,28,33,36,40,45,37,41,52,46,43]:
            return 'Medium_Age_Market'
        else:
            return 'New_Market'

    def signtobook(x):
        if x <=0 :
            return 0
        elif x>=90:
            return 90
        else:
            return x
    ### categorical transformation 

    if Train:
        data['client_rank_label'] = data['rank'].apply(cat_client_rank)

    data['booking_hour_ahead1'] = data['hours_ahead1'].apply(cat_booking_hour)
    data['booking_hour_ahead2'] = data['hours_ahead2'].apply(cat_booking_hour)
    data['market_maturity'] = data['city_id'].apply(cat_city_maturity)
    data['signup_to_book_interval'] = data['sign_up_to_book'].apply(signtobook)
    data['booking_interval']=data['interval_between_bookings'].apply(booking_interval)
    #data['session_length_type'] = data['session_length'].apply(cat_session_length)
    data['income_level'] = data.median_income
    ### drop columns
    column_drop = ['user_id', 'sign_up_to_book', 'hours_ahead1','hours_ahead2',  'city_id', 'median_income',\
                   'rank','interval_between_bookings']
    data_merge = data.drop(column_drop, axis = 1)
    ### create dummy variable
    column_dummy = ['booking_hour_ahead1','booking_hour_ahead2', 'market_maturity']
    data_train_dummies = pd.get_dummies(data_merge, columns = column_dummy,drop_first=True)
    ### make categorical variable n-1
    #baseline = ['booking_hour_ahead_Adv_Booked','market_maturity_New_Market','session_length_type_short_session']
    #data_train_dummies_remove_base = data_train_dummies.drop(baseline, axis = 1)
    return data_train_dummies,data['user_id']

### data scaling
def min_max_scale(train_data, test_data):
    sc = MinMaxScaler()
    scale_list = ['signup_to_book_interval','booking_interval','quality_score1','quality_score2','quality_score_diff',\
                  'revenue1','revenue2',\
                  'income_level','num_hvc']
    data_train_dummies_remove_base_scaled_train = train_data.loc[:,[s for s in train_data.columns if s not in scale_list]]
    data_train_dummies_remove_base_scaled_test = test_data.loc[:,[s for s in test_data.columns if s not in scale_list]]
    for candidate_column in scale_list:
        data_train_dummies_remove_base_scaled_train[candidate_column] = sc.fit_transform(train_data.loc[:,candidate_column])  
        data_train_dummies_remove_base_scaled_test[candidate_column] = sc.transform(test_data.loc[:,candidate_column])
    return data_train_dummies_remove_base_scaled_train,data_train_dummies_remove_base_scaled_test

### model fit

def get_params(model, param_list):
    model_name = model.__class__.__name__
    params = param_list.get(model_name, {})
    return params


def model_fit(model, init, param_list, X_train, y_train):
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    if init:
        params = get_params(model,param_list)
        model.set_params(**params)
    model.fit(X_train,y_train)
    return model

def auc_cross_validate(model,X_train,y_train):
    X_train = np.array(X_train)
    y_train = np.array(y_train).reshape(-1)
    skf = StratifiedKFold(n_splits = 3, random_state = 1234, shuffle = False)
    score = []
    for train_index,test_index in skf.split(X_train,y_train):
        train_data_x,test_data_x = X_train[train_index],X_train[test_index]
        train_data_y,test_data_y = y_train[train_index],y_train[test_index]
        model.fit(train_data_x,train_data_y)
        pred = model.predict_proba(test_data_x)[:,1]
        score.append(metrics.roc_auc_score(test_data_y,pred))
    return (np.mean(score))
    


def find_best_params(model,X_train,y_train,PARAM_GRID,cv,scoring,verbose):
    X_train = np.array(X_train)
    y_train = np.array(y_train).reshape(-1)
    gv = GridSearchCV(model, PARAM_GRID[model.__class__.__name__], cv = cv, scoring = scoring, verbose = verbose)
    gv.fit(X_train,y_train)
    params = model.get_params()
    params.update(gv.best_params_)
    model.set_params(**params)
    auc = auc_cross_validate(model,X_train,y_train)
    return params,auc

### save and load model

def save_model(model):
    model_name = model.__class__.__name__
    save_path = os.getcwd()+'/'+model_name+'.pkl'
    if os.path.exists(save_path):
        os.remove(save_path)
    outfile = open(save_path,'wb')
    try:
        pickle.dump(model,outfile)
        outfile.close()
    except Exception as e:
        print("Error in save_model")
        print(e)
        #newrelic.agent.record_exception(e)

    #s3_connection.upload_file(file_name = model_name+".pkl")
    
def save_pred(filename,preds):
    save_path=os.path.join(os.getcwd(),filename+'.pkl')
    if os.path.exists(save_path):
        os.remove(save_path)
    outfile=open(save_path,'wb')
    try:
        pickle.dump(preds,outfile)
        outfile.close()
    except Exception as e:
        print('Error in save_pred')
        print(e)
    

def current_model(**save_path):
    try:
        if save_path:
            pass
        else:
            save_path = os.getcwd()
        for file in os.listdir(save_path):
            if file.endswith('.pkl'):
                print (file)
    except:
        print ('Invaid Path')

def load_model(model_name):
    try:
        #s3_connection.download_file(model_name+".pkl")
        load_path = os.getcwd()+'/'+model_name+'.pkl'
        infile = open(load_path,'rb')
        model = pickle.load(infile)
        infile.close()
        print('Model Loaded Succesfully.')
        return model
    except Exception as e:
        print("Error in load_model")
        print(e)
        #newrelic.agent.record_exception(e)

def save_threshold(threshold_dict):
    model_name = "threshold_dict.pkl"
    save_path = os.getcwd()+'/' + model_name
    if os.path.exists(save_path):
        os.remove(save_path)
        print('Old File Removed.')
    
    try:
        outfile = open(save_path,'wb')
        pickle.dump(threshold_dict,outfile)
        outfile.close()
        print ('Saved Threshold Dict Successfully.')
        #s3_connection.upload_file(file_name = model_name)
    except:
        print('Failed to Save the Threshold Dict.')

def load_threshold():
    pickle_file = "threshold_dict.pkl"
    try:
        #s3_connection.download_file(pickle_file)
        load_path = os.getcwd()+'/' + pickle_file
        infile = open(load_path,'rb')
        threshold_dict = pickle.load(infile)
        infile.close()
        print('Threshold Dict Loaded Succesfully.')
        return threshold_dict
    except Exception as e:
        print("Error in load_threshold")
        print(e)


### select best threshold

def best_threshold_cv(model,X_train,y_train):
    best_threshold = 0
    best_f1_score = -1
    best_recall = 0
    best_accuracy_score = 0
    skf = StratifiedKFold(n_splits=3,random_state=1234,shuffle=False)
    X_train = np.array(X_train)
    y_train = np.array(y_train).reshape(-1)    

    for threshold in np.arange(0.001,0.999,0.001):
        f1_score = []
        recall_score = []
        accuracy_score = []
        for train_index,test_index in skf.split(X_train,y_train):
            test_data_x = X_train[test_index]
            test_data_y = y_train[test_index]
            p1 = np.where(model.predict_proba(test_data_x)[:,1]>=threshold,1,0)
            f1_score.append(metrics.f1_score(test_data_y,p1))
            recall_score.append(metrics.recall_score(test_data_y,p1))
            accuracy_score.append(metrics.accuracy_score(test_data_y,p1))
        current_score = np.mean(f1_score)
        current_recall = np.mean(recall_score)
        current_accuracy_score = np.mean(accuracy_score)
        if (current_score >= best_f1_score) & (current_recall >= 0.60):
            best_f1_score = current_score
            best_threshold = threshold
            best_recall = current_recall
            best_accuracy_score = current_accuracy_score

    return best_threshold,best_accuracy_score,best_f1_score,best_recall



In [38]:
train_data_preprocess.to_csv('processed_data.csv',index=False)

In [51]:
x_Train=train_data_preprocess.loc[:,train_data_preprocess.columns!='client_rank_label']

In [52]:
y_Train=train_data_preprocess.loc[:,train_data_preprocess.columns=='client_rank_label']

In [53]:
Random_State=1234
x_train_not_scaled,x_test_not_scaled,y_train,y_test=train_test_split(x_Train,y_Train,test_size=0.2,random_state=Random_State)

In [56]:
x_train,x_test=min_max_scale(x_train_not_scaled,x_test_not_scaled)

In [59]:
clf_lr=LogisticRegression(penalty='l1',random_state=Random_State)

In [60]:
clf_lr.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [86]:
print(auc_cross_validate(model=clf_lr,X_train=x_train,y_train=y_train))

0.77038942037


In [89]:
print(best_threshold_cv(clf_lr,x_train,y_train))

(0.159, 0.76306818181818181, 0.33232101191522345, 0.5770775409329626)


In [99]:
list(INIT_GRID.keys())

['DecisionTreeClassifier',
 'ExtraTreesClassifier',
 'XGBClassifier',
 'RandomForestClassifier',
 'LogisticRegression']

In [ ]:
AdaBoostClassifier:{'learning_rate':0.1,'random_state':Random_State}

In [8]:
a={'a':1,'b':2,'c':3}
sorted(a.items(),key=operator.itemgetter(1),reverse=True)[0][0]

'c'

In [138]:
train_filename='traindata.csv'
test_filename='testdata.csv'
Random_State=12345
# initiate model 
INIT_GRID={'LogisticRegression':{'C':0.1,'penalty':'l1','class_weight': 'balanced'},
       'RandomForestClassifier':{'n_estimators':500,'max_depth':10,'max_features':.1,'min_samples_leaf':2,'random_state':Random_State},
        'XGBClassifier':{'base_score': 0.5,'colsample_bylevel': 1,'colsample_bytree': 1,'gamma': 0,'learning_rate': 0.1,'max_delta_step': 0,'max_depth': 3,'min_child_weight': 1,'missing': None,'n_estimators': 10,'nthread': -1,
        'objective': 'binary:logistic','reg_alpha': 0,'reg_lambda': 1,'scale_pos_weight': 1,'seed': 0,'silent': True,'subsample': 1},
        'ExtraTreesClassifier':{'n_estimators':500,'max_depth':10,'max_features':.1,'min_samples_leaf':10,'random_state':Random_State},
        'DecisionTreeClassifier': {'max_features':0.8,'max_depth':10,'min_samples_leaf':10,'random_state':Random_State},
        'AdaBoostClassifier':{'learning_rate':0.1,'random_state':Random_State}
       }

PARAM_GRID={'LogisticRegression':{'C':[0.0001,0.001,0.01,0.1],'class_weight': ['balanced',None],'penalty':['l1','l2']},
       'RandomForestClassifier':{'max_depth':[3,5,10],'max_features':['auto','sqrt'],'min_samples_leaf':[2,4,6,8]},
        'XGBClassifier':{'max_depth':[3,5,10],'learning_rate':[0.01,0.1,0.5,1],'min_child_weight':range(1,6,2),'gamma':[i/10.0 for i in range(0,5)]},
        'ExtraTreesClassifier':{'max_depth':[3,5,10],'max_features':['auto','sqrt'],'min_samples_leaf':[2,4,6,8]},
        'DecisionTreeClassifier': {'max_features':['auto','sqrt'],'max_depth':[3,5,10],'min_samples_leaf':[2,4,6,8]},   
        'AdaBoostClassifier':{'learning_rate':[0.01,0.1,1.],'n_estimators':[10,50,100]}
       }

# preprocess the training set
train_data_preprocess,train_data_user_id=data_preprocessing(train_filename,Train=True)
x_Train_not_scaled=train_data_preprocess.loc[:,train_data_preprocess.columns!='client_rank_label']
y_Train=train_data_preprocess.loc[:,train_data_preprocess.columns=='client_rank_label']
x_train_not_scaled,x_test_not_scaled,y_train,y_test=train_test_split(x_Train_not_scaled,y_Train,test_size=0.2,random_state=Random_State)
x_train,x_test=min_max_scale(x_train_not_scaled,x_test_not_scaled)

# preprocess the testing set
test_data_preprocess,test_data_user_id=data_preprocessing(test_filename,Train=False)
x_Test_not_scaled=test_data_preprocess.loc[:,test_data_preprocess.columns!='client_rank_label']
x_Train,x_Test=min_max_scale(train_data=x_Train_not_scaled,test_data=x_Test_not_scaled)



In [35]:
def get_model_cv_preds(model,X_train,y_train,random_state):
    X_train=np.array(X_train)
    y_train=np.array(y_train).reshape(-1)
    model_name=model.__class__.__name__
    clf=load_model(model_name)
    skf=StratifiedKFold(n_splits=3,random_state=random_state,shuffle=False)
    stack_preds=list()
    cv_index=list()
    for train_index,test_index in skf.split(X_train,y_train):
        train_data_x,test_data_x=X_train[train_index],X_train[test_index]
        train_data_y,test_data_y=y_train[train_index],y_train[test_index]
        clf.fit(train_data_x,train_data_y)
        stack_preds.extend(list(clf.predict_proba(test_data_x)[:,1]))
        #stack_preds.extend(list(test_data_y))
        cv_index.extend(test_index)
    stack_preds=np.array(stack_preds)[sp.argsort(cv_index)]
    return stack_preds

In [43]:
stack_preds=get_model_cv_preds(XGBClassifier(),X_train=x_train,y_train=y_train,random_state=Random_State)

Model Loaded Succesfully.


In [44]:
metrics.roc_auc_score(y_train,stack_preds)

0.83155759053596423

In [139]:
def stacking(models,X_train,y_train,X_test,test_user_id,random_state,train,prediction,PARAM_GRID):
    if train:
        generalizer=LogisticRegression()
        stage0_train=list()
        #stage0_predict=list()
        for model in models:
            #model.fit(np.array(X_train),np.array(y_train).reshape(-1))
            #stage0_predict.append(model.predict_proba(np.array(X_train))[:,1])
            stage0_train.append(get_model_cv_preds(model,X_train,y_train,random_state))
        skf=StratifiedKFold(n_splits=3,shuffle=False,random_state=Random_State)
        best_g_params,best_g_auc=find_best_params(model=generalizer,cv=skf,PARAM_GRID=PARAM_GRID,scoring='roc_auc',verbose=1,X_train=np.array(stage0_train).T,y_train=y_train)
        print('Best Generalizer Parms: ',best_g_params)
        print('Best Generalizer AUC: ',best_g_auc)
        generalizer.set_params(**best_g_params)
        generalizer.fit(np.array(stage0_train).T,np.array(y_train).reshape(-1))
        save_model(generalizer)
        print('''
        ###################################
        Finished Training the Stacked Model
        ###################################''')
        
        best_threshold,best_accuracy_score,best_f1_score,best_recall=best_threshold_cv(generalizer,X_train=np.array(stage0_train).T,y_train=np.array(y_train).reshape(-1))
        threshold_dict={'Best_Threshold':best_threshold,'Best_Accuracy_Score':best_accuracy_score,'Best_F1_Score':best_f1_score,'Best_Recall':best_recall}
        print('Best Threshold: ',best_threshold,'Best Accuarcy Score: ',best_accuracy_score,'Best F1 Score: ',best_f1_score,'Best Recall: ',best_recall)
        save_threshold(threshold_dict)
        print('''
        ###############################
        Finished Training the Threshold
        ###############################''')
    if prediction:
        generalizer=load_model('LogisticRegression')
        threshold_dict=load_threshold()
        print('Best Threshold: ', threshold_dict['Best_Threshold'])
        stage1_test=list()
        for model in models:
            model_pred=load_model(model.__class__.__name__)
            model_pred.fit(np.array(X_train),np.array(y_train).reshape(-1))
            stage1_test.append(list(model_pred.predict_proba(np.array(X_test))[:,1]))
        
        #return generalizer.predict_proba(np.array(stage1_test).T)[1:]
        print (np.array(stage1_test).shape)
        prob=generalizer.predict_proba(np.array(stage1_test).T)[:,1]
        preds=np.where(prob>=threshold_dict['Best_Threshold'],1,0)
        output=pd.DataFrame({'user_id':test_user_id,'score':prob,'status':preds},columns=['user_id','score','status'])
        if os.path.exists('predict.csv'):
            os.remove('predict.csv')
        output.to_csv('predict.csv',index=False)
        print('''
        ########################
        Done running prediction
        ########################''')
    
        
        

In [140]:
stacking(models=models,X_train=x_train,y_train=y_train,X_test=x_Test,test_user_id=test_data_user_id,PARAM_GRID=PARAM_GRID,train=True,prediction=True,random_state=Random_State)

Model Loaded Succesfully.
Model Loaded Succesfully.
Model Loaded Succesfully.
Model Loaded Succesfully.
Model Loaded Succesfully.
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    9.6s finished


Best Generalizer Parms:  {'fit_intercept': True, 'multi_class': 'ovr', 'tol': 0.0001, 'dual': False, 'n_jobs': 1, 'solver': 'liblinear', 'verbose': 0, 'class_weight': 'balanced', 'max_iter': 100, 'random_state': None, 'intercept_scaling': 1, 'penalty': 'l1', 'warm_start': False, 'C': 0.1}
Best Generalizer AUC:  0.833793107172

        ###################################
        Finished Training the Stacked Model
        ###################################
Best Threshold:  0.614 Best Accuarcy Score:  0.814235927655 Best F1 Score:  0.400662332699 Best Recall:  0.60281462516
Old File Removed.
Saved Threshold Dict Successfully.

        ###############################
        Finished Training the Threshold
        ###############################
Model Loaded Succesfully.
Threshold Dict Loaded Succesfully.
Best Threshold:  0.614
Model Loaded Succesfully.
Model Loaded Succesfully.
Model Loaded Succesfully.
Model Loaded Succesfully.
Model Loaded Succesfully.
(5, 305)

        ##############

In [109]:
x_Test.shape

(4289, 26)

In [99]:
[x for x in x_train.columns if x not in x_Test.columns]

['prime_session1', 'prime_session2', 'late_session1', 'late_session2']

In [52]:
save_pred('stage0_train',stack_preds)

In [53]:
load_model_list=['RandomForestClassifier','XGBClassifier','ExtraTreesClassifier','DecisionTreeClassifier','AdaBoostClassifier']

In [80]:
models=[RandomForestClassifier(),XGBClassifier(),ExtraTreesClassifier(),DecisionTreeClassifier(),AdaBoostClassifier()]

In [120]:
stacking(models=models,PARAM_GRID=PARAM_GRID,train=True,test_user_id=[],prediction=False,X_test=[],X_train=x_train,y_train=y_train,random_state=Random_State)

TypeError: stacking() missing 1 required positional argument: 'test_user_id'

In [70]:
train_data_preprocess.iloc[:,:-1].columns

Index(['diff_zip', 'quality_score1', 'quality_score2', 'quality_score_diff',
       'revenue1', 'revenue2', 'discount_ratio1', 'discount_ratio2',
       'prime_session1', 'prime_session2', 'late_session1', 'late_session2',
       'couple1', 'couple2', 'weekend1', 'weekend2', 'special_request2',
       'num_hvc', 'client_rank_label', 'signup_to_book_interval',
       'booking_interval', 'income_level', 'booking_hour_ahead1_On_Demand',
       'booking_hour_ahead2_On_Demand', 'market_maturity_Mature_Market',
       'market_maturity_Medium_Age_Market'],
      dtype='object')

In [72]:
train_data_preprocess.iloc[:,:].columns

Index(['diff_zip', 'quality_score1', 'quality_score2', 'quality_score_diff',
       'revenue1', 'revenue2', 'discount_ratio1', 'discount_ratio2',
       'prime_session1', 'prime_session2', 'late_session1', 'late_session2',
       'couple1', 'couple2', 'weekend1', 'weekend2', 'special_request2',
       'num_hvc', 'client_rank_label', 'signup_to_book_interval',
       'booking_interval', 'income_level', 'booking_hour_ahead1_On_Demand',
       'booking_hour_ahead2_On_Demand', 'market_maturity_Mature_Market',
       'market_maturity_Medium_Age_Market', 'market_maturity_New_Market'],
      dtype='object')

In [142]:
parser = argparse.ArgumentParser()
help(parser.add_argument)

Help on method add_argument in module argparse:

add_argument(*args, **kwargs) method of argparse.ArgumentParser instance
    add_argument(dest, ..., name=value, ...)
    add_argument(option_string, option_string, ..., name=value, ...)



In [146]:
import time

start_time=time.time()
end_time=time.time()
print("It took: %.2f"%(end_time-start_time))

It took: 0.00
